In [ ]:
# Generate graphics for the datasets generated in Redis Instant Recovery
# The dataset is a CSV with the fields key,command,startTime,finishTime and type. Each field represents a commando execution on database.
# The graphics shows the database throughput (number of commands executed in a period of time) over time

#path = path to dataset
def processData(path):
    data = open(path).readlines()
    
    ln = data[1].split(",")
    initialTime = int(ln[3])
    
    ln = data[2].split(",")
    recoveryTime = int(ln[3])
    
    ln = data[3].split(",")
    bechmarkTime = int(ln[3])
    
    #Removes header lines and possible zero times or corrupted lines
    data.pop(0)
    data.pop(0)
    data.pop(0)
    data.pop(0)
    
    total = 0
    # Gets and converts the items in column endTime (in microseconds) from the 
    # data to seconds and, converts to relative time (begining from zero). 
    x2 = []
    for i in range(len(data)):
            total += 1
            linha = data[i].split(",")
            if linha[3].isnumeric():#prevents to a possible corrupted line
                num = int(linha[3]) 
                x2.append(int((num - initialTime)/1000000))
    
    # Creates a list (x) of disticts numbers (end times) and another (y) of zeros.
    x = []
    y = []
    x.append(x2[0])
    current = x2[0]
    y.append(0)
    
    for i in range(len(x2)):
        if current != x2[i]:
            x.append(x2[i])
            current = x2[i]
            y.append(0)
    
    # Counts the number operations (y) that ends in the same time (x). This is the
    # throughput (transaction per second).
    j = 0
    for i in range(len(x2)):
        if x[j] == x2[i]:
            y[j] += 1
        else:
            j += 1
        
    print("Total of lines from dataset:", total, ", initialTime = ", initialTime)
    
    print( "Average throughput = " , sum(y) / float(len(y)) )
    
    if recoveryTime != -1:
        print("Recovery time = ", recoveryTime, ". ")
    else:
        print("Recovery time was not stored! ")
    if bechmarkTime != -1:
        print("Bechmark time = ", bechmarkTime, ". ")
    else:
        print("Bechmark time was not stored!")
        
    print("X lines = ", len(x), " - Y lines = ", len(y))
    
    return x, y, recoveryTime

In [ ]:
#Get the x (time) and y (throughput)
x_data = []
y_data = []

x_data, y_data, recTime = processData('src/datasets/datasets.csv')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14,7))

font_size2 = 14
font_size = 18

plt.plot(x_data, y_data, label="AILIR - Asynchronous Indexed Log Instant Recovery", color = 'blue')
plt.axvline(x=recTime, linestyle='dashed', color = 'blue')
c = []
d = []
c.append(recTime)
d.append(14000)
plt.scatter(c, d, color = 'blue')
plt.text(recTime-60,14000,'AILIR recovery time → '+str(recTime), color = 'blue', fontsize=font_size2)

plt.xticks(fontsize=font_size)
plt.yticks(fontsize=font_size)
plt.xlabel("Time (seconds)", fontsize=font_size)
plt.ylabel("Transaction Throughput", fontsize=font_size)
plt.legend(loc="best",  prop={'size':font_size2})

plt.show()